In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import imdb

- IMDB 파일은 영화 리뷰 문장이며, 긍정 리뷰일 경우 1, 부정 리뷰일 경우 label은 0임

In [2]:
dataset = np.load('imdb.npz',allow_pickle=True)
train_sequences = dataset['arr_0']
train_labels = dataset['arr_1']
test_sequences = dataset['arr_2']
test_labels = dataset['arr_3']

## ※ 단어 사전에 포함된 데이터의 수는 500개 (num_words = 500)

### 1. 로드한 train_sequences, train_labels, test_sequences, test_labels의 타입을 확인하시오.

In [3]:
type(train_sequences), type(train_labels), type(test_sequences), type(test_labels)

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

### 2. training 데이터와 test 데이터의 수를 확인하시오.

In [4]:
print(train_sequences.shape, train_labels.shape, test_sequences.shape, test_labels.shape)

(5000,) (5000,) (1000,) (1000,)


[정답] training_data는 5000개, test_data는 1000개이다.

### 3. 문장의 최대 길이를 100단어로 설정하고, 긴 문장의 뒷단어를 삭제하고, 짧은 단어의 뒤를 0으로 채우시오.

- 변환된 데이터를 trainPad, testPad에 저장하시오.

In [10]:
# 코드 작성
trainPad = pad_sequences(train_sequences, maxlen = 100, truncating = 'post', padding = 'post')
testPad = pad_sequences(test_sequences, maxlen = 15, truncating = 'post', padding = 'post')

### 4. 단어를 one hot encoding 했을 경우, 한 문장은 몇 개의 수(0 또는 1)로 구성되는가?

- 최대 문장 길이와 사전에 포함된 단어의 수를 고려하시오.

[정답] 각 문자를 표현하기 위해 500개의 수가 필요하므로 한 문장은 총 50,000개의 수로 구성되어있다.

### 5. 한 단어의 수를 10개의 실수로 변환하는 embedding layer를 포함한 RNN(LSTM)을 구성하시오.

In [14]:
# 코드 완성
model_NLP = Sequential([
    Embedding(500, 10, input_length=100),   
    Bidirectional(LSTM(64, return_sequences = True)),
    Bidirectional(LSTM(32)),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')
])

### 6. LSTM의 특성을 간단히 서술하시오.

[정답] RNN 모델에 비교하여 기울기 손실(vanishing gradient)을 줄여줄 수 있다는 장점이 있습니다. 시계열 데이터 처리 및 자연어 처리와 같이 앞선 작업이 뒷 작업에 영향을 주는 모델에 사용하기 적합합니다.

### 7. 모델을 compile하고 training data를 이용하여 학습하시오.
- validataion을 위한 데이터 비율을 30%로 설정하시오.
- epoch은 10으로 설정하되, 다른 파라미터는 자유롭게 선정하시오.

In [15]:
# 코드 작성
model_NLP.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

result = model_NLP.fit(trainPad, train_labels, epochs = 10, batch_size=256, validation_split = 0.3)

Epoch 1/10
14/14 [==============================] - 5s 227ms/step - loss: 0.6928 - acc: 0.5069 - val_loss: 0.6939 - val_acc: 0.5033
Epoch 2/10
14/14 [==============================] - 3s 214ms/step - loss: 0.6900 - acc: 0.5363 - val_loss: 0.6897 - val_acc: 0.5527
Epoch 3/10
14/14 [==============================] - 3s 208ms/step - loss: 0.6795 - acc: 0.5863 - val_loss: 0.6636 - val_acc: 0.6007
Epoch 4/10
14/14 [==============================] - 3s 218ms/step - loss: 0.6413 - acc: 0.6369 - val_loss: 0.6415 - val_acc: 0.6307
Epoch 5/10
14/14 [==============================] - 3s 223ms/step - loss: 0.5930 - acc: 0.6929 - val_loss: 0.6455 - val_acc: 0.6200
Epoch 6/10
14/14 [==============================] - 3s 216ms/step - loss: 0.5452 - acc: 0.7346 - val_loss: 0.5730 - val_acc: 0.7033
Epoch 7/10
14/14 [==============================] - 3s 215ms/step - loss: 0.4972 - acc: 0.7683 - val_loss: 0.5546 - val_acc: 0.7313
Epoch 8/10
14/14 [==============================] - 3s 215ms/step - loss: 0.

### 8. test data에 대한 성능을 평가하시오. (정확도를 표시할 것)

In [17]:
model_NLP.evaluate(testPad, test_labels)

32/32 [==============================] - 0s 2ms/step - loss: 0.6652 - acc: 0.5960


[0.6651767492294312, 0.5960000157356262]

In [18]:
# 정확도 : 0.5960000157356262